In [2]:
# import the packages
import pandas as pd
import numpy as np
import pickle
import glob,os

In [117]:
rawdf = pd.read_csv('follower-tst-xinlitai.csv')

In [6]:
rawdf.head()

,标题,用户_链接,用户,字段,页面网址,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,白马大侠,https://xueqiu.com/4224679077,24913 粉丝|120 关注|2175 帖子,价值选股，趋势定仓,https://xueqiu.com/u/6405911882#/follow,NaN,NaN,NaN,NaN
1,百创资本陈子仪,https://xueqiu.com/4403644081,33110 粉丝|172 关注|155 帖子,我从2001年开始投资，2001年到2008年主要投资A股B股，2009年开始专注港股和美股...,https://xueqiu.com/u/6405911882#/follow,NaN,NaN,NaN,NaN
2,木文田,https://xueqiu.com/1621361008,13871 粉丝|164 关注|665 帖子,证券市场里的钱是一种很骄傲的东西，它会用深入骨髓的眼光透视你灵魂深处的弱点，然后化身出最诱惑...,https://xueqiu.com/u/6405911882#/follow,NaN,NaN,NaN,NaN
3,价值事务所,https://xueqiu.com/1859139457,37699 粉丝|48 关注|1034 帖子,微信公众号：价值事务所,https://xueqiu.com/u/6405911882#/follow,NaN,NaN,NaN,NaN
4,青侨阳光,https://xueqiu.com/houqiang,51157 粉丝|233 关注|292 帖子,聚焦医药。谦逊而坚韧，冗余而穿透。,https://xueqiu.com/u/6405911882#/follow,NaN,NaN,NaN,NaN


In [129]:
# 获取其中的follwing和用户
# 只保留其中的两列，一列是用户连接 一列页面网址，用户的粉丝数和关注数帖子数留存下来，拆分开作为备用数据，标题可以去掉
'''
del rawdf['标题']
del rawdf['字段']
del rawdf['Unnamed: 5']
del rawdf['Unnamed: 6']
del rawdf['Unnamed: 7']
del rawdf['Unnamed: 8']
'''

"\ndel rawdf['标题']\ndel rawdf['字段']\ndel rawdf['Unnamed: 5']\ndel rawdf['Unnamed: 6']\ndel rawdf['Unnamed: 7']\ndel rawdf['Unnamed: 8']\n"

In [120]:
rawdf.head()

,用户_链接,用户,页面网址
0,https://xueqiu.com/4224679077,24913 粉丝|120 关注|2175 帖子,https://xueqiu.com/u/6405911882#/follow
1,https://xueqiu.com/4403644081,33110 粉丝|172 关注|155 帖子,https://xueqiu.com/u/6405911882#/follow
2,https://xueqiu.com/1621361008,13871 粉丝|164 关注|665 帖子,https://xueqiu.com/u/6405911882#/follow
3,https://xueqiu.com/1859139457,37699 粉丝|48 关注|1034 帖子,https://xueqiu.com/u/6405911882#/follow
4,https://xueqiu.com/houqiang,51157 粉丝|233 关注|292 帖子,https://xueqiu.com/u/6405911882#/follow


In [121]:
# 先分割在进行数据合并
rawdf1 = rawdf['用户'].str.split('|',expand=True)
#print(rawdf1.head())
newnames = ['fans_num','following_num','posts_num']
rawdf1.columns = newnames
#print(rawdf1.head())
# process the fans number and the following number
rawdf['fans_num']  = rawdf1['fans_num'].str.split(' ',expand=True)[0]
rawdf['following_num']  = rawdf1['following_num'].str.split(' ',expand=True)[0]
rawdf['posts_num']  = rawdf1['posts_num'].str.split(' ',expand=True)[0]
rawdf['following_uid']  = rawdf['用户_链接'].str[19:]
rawdf['crowd_member_uid']  = rawdf['页面网址'].str[21:].str.split('#',expand=True)[0]
del rawdf['用户_链接']
del rawdf['用户']
del rawdf['页面网址']
print(rawdf.head())


  fans_num following_num posts_num following_uid crowd_member_uid
0    24913           120      2175    4224679077       6405911882
1    33110           172       155    4403644081       6405911882
2    13871           164       665    1621361008       6405911882
3    37699            48      1034    1859139457       6405911882
4    51157           233       292      houqiang       6405911882


In [122]:
order = ['crowd_member_uid','following_uid','fans_num','following_num','posts_num']
followingls_df = rawdf[order]
print(followingls_df.head())

  crowd_member_uid following_uid fans_num following_num posts_num
0       6405911882    4224679077    24913           120      2175
1       6405911882    4403644081    33110           172       155
2       6405911882    1621361008    13871           164       665
3       6405911882    1859139457    37699            48      1034
4       6405911882      houqiang    51157           233       292


In [128]:
#再建立一张表是crowd members
cwdf = pd.read_csv('xinlitai_crowd_member.csv')
#print(cwdf)
members_ls =cwdf["crowd_members"].tolist()
#print(members_ls)
#len(members_ls)

In [132]:
# create the network
cw_netdf = followingls_df.loc[followingls_df['following_uid'].isin(members_ls)]
# 如果没有联系的crowd如何加进去呢？

In [157]:
# 原始数据表处理
# 将不同公司的数据表分开，同时合并文件进行ml的训练
# following和follower一起采集
commentdf = pd.read_csv('stockcomment-tst.csv')
#commentdf.head()
# 先要将表分开，将各个股票自己成为一个表
# 对表格进行处理，将用户和股票代码提取出来
stock_cwms =  commentdf[['文本','用户_链接']]
stock_cwms1 = stock_cwms['文本'].str.split('(',expand=True)
stock_cwms1.columns = ['stock_name','stock_code']
#print(stock_cwms1)
stock_cwms['members'] = stock_cwms['用户_链接'].str[19:]
stock_cwms['stock_name'] = stock_cwms1['stock_name']
stock_cwms['stock_code'] = stock_cwms1['stock_code'].str[:9]
del stock_cwms['文本']
del stock_cwms['用户_链接']


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


      members stock_name stock_code
0  6405911882        信立泰  SZ:002294
1  8890117734        信立泰  SZ:002294
2  8960894035        信立泰  SZ:002294
3  6405911882        信立泰  SZ:002294
4  8960894035        信立泰  SZ:002294


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [163]:
#print(stock_cwms.head())
df1 = stock_cwms.loc[stock_cwms['stock_code'] == 'SZ:002294']
print(df1)

        members stock_name stock_code
0    6405911882        信立泰  SZ:002294
1    8890117734        信立泰  SZ:002294
2    8960894035        信立泰  SZ:002294
3    6405911882        信立泰  SZ:002294
4    8960894035        信立泰  SZ:002294
5    8960894035        信立泰  SZ:002294
6    8960894035        信立泰  SZ:002294
7    9956679466        信立泰  SZ:002294
8    5244250798        信立泰  SZ:002294
9    8960894035        信立泰  SZ:002294
10   2054567330        信立泰  SZ:002294
11   9956679466        信立泰  SZ:002294
12   4255434670        信立泰  SZ:002294
13   8960894035        信立泰  SZ:002294
14   9956679466        信立泰  SZ:002294
15   1597977269        信立泰  SZ:002294
16   9956679466        信立泰  SZ:002294
17   5896904950        信立泰  SZ:002294
18   9956679466        信立泰  SZ:002294
19   8960894035        信立泰  SZ:002294
20   8801386347        信立泰  SZ:002294
21   8890117734        信立泰  SZ:002294
22   8801386347        信立泰  SZ:002294
23   4365015103        信立泰  SZ:002294
24   8960894035        信立泰  SZ:002294
25   4365015

In [88]:

# pkl包系列，用于数据的处理和读取
# code source: https://zhuanlan.zhihu.com/p/104492767?from_voters_page=true
def save_pkl(path,obj):
    pickle_file = open(path,'wb')
    pickle.dump(obj, pickle_file)
    pickle_file.close()
    print("保存成功")
def load_pkl(path):
    pickle_file = open(path,'rb')
    obj=pickle.load(pickle_file)
    pickle_file.close()
    print("读取成功")
    return obj

In [ ]:
# data preprocess:评论表数据处理
# 处理后的数据表应该长成什么样？
# 然后要得到每一个股票的unique member uid从而进行检索关注者和被关注的人
# 然后计算diversity


In [79]:
# import data 
path=r'./popcmdata' # the location of the dataset 
file=glob.glob(os.path.join(path, "popularposts1-150*.csv")) # filename
print(file)

# to concat the dataframes
dl= []
for f in file:
    dl.append(pd.read_csv(f))
stock_commentsp1 =pd.concat(dl,axis = 0, ignore_index = True) 

['./popcmdata\\popularposts1-150(1).csv', './popcmdata\\popularposts1-150(2).csv', './popcmdata\\popularposts1-150(3).csv', './popcmdata\\popularposts1-150(4).csv', './popcmdata\\popularposts1-150(5).csv', './popcmdata\\popularposts1-150(6).csv', './popcmdata\\popularposts1-150(7).csv', './popcmdata\\popularposts1-150(8).csv']


In [82]:
# data preprocess：评论数据总表处理
def raw_dataprocess(rawdf):
    '''
    input: the data being processed
    output: processed new dataframe
    '''
    # get the stock code
    rawdf['stock_code'] = rawdf['页面网址'].str[21:29]
    # get the crowd member uid
    rawdf['user_uid'] =  rawdf['用户_链接'].str[19:]
    # change the column name
    rawdf.rename(columns={'用户':'user_name', '内容':'content','来源':'post_time','时间':'retweet_num','时间2':'reply_num','时间2':'reply_num','时间3':'like_num'}, inplace = True)
    rawdf_new =  rawdf[['stock_code','user_name','user_uid','post_time','content','retweet_num','reply_num','like_num']] 
    # extract numbers from several columns
    cg_ls = ['retweet_num','reply_num','like_num']
    for i in cg_ls:
        rawdf_new[i] = rawdf_new[i].str.extract(r'(\d+)')
    # split the comment time
    rawdf_new['post_md'] = rawdf_new['post_time'].str.split(' ',expand = True).iloc[:,0]
    rawdf_new['post_hm'] = rawdf_new['post_time'].str.split(' ',expand = True).iloc[:,1]
    rawdf_new1 = rawdf_new[['stock_code','user_name','user_uid','post_md','post_hm','content','retweet_num','reply_num','like_num']]
    return  rawdf_new1

In [84]:
stock_com = raw_dataprocess(stock_commentsp1)

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [93]:
# data preprocess:股票分割
def stockdf_split(rawdf,colindex):
    '''
    input:数据表和code所在列的Index
    output:将分割后的数据表保存在本地
    需要save pkl函数
    此数据不局限于某一特定的数据表
    '''
    code_ls = rawdf[colindex].unique().tolist()
    num = 0
    for cds in code_ls:
        num += 1
        df = rawdf.loc[rawdf[colindex] == cds]
        outfilename = cds + '.csv'
        df.to_csv(outfilename,encoding="utf_8_sig") # solve the encoding issue
        print('第'+str(num)+"个文件保存成功，股票代码为"+cds)
        

In [94]:
stockdf_split(stock_com,'stock_code')

第1个文件保存成功，股票代码为SH601066
第2个文件保存成功，股票代码为SZ002050
第3个文件保存成功，股票代码为SZ002294
第4个文件保存成功，股票代码为SZ300059
第5个文件保存成功，股票代码为SH601881
第6个文件保存成功，股票代码为SH603501
第7个文件保存成功，股票代码为SH601727
第8个文件保存成功，股票代码为SZ300033
第9个文件保存成功，股票代码为SZ000858
第10个文件保存成功，股票代码为SH601788
第11个文件保存成功，股票代码为SZ002044
第12个文件保存成功，股票代码为SH603259
第13个文件保存成功，股票代码为SH600655
第14个文件保存成功，股票代码为SZ300347
第15个文件保存成功，股票代码为SZ300142
第16个文件保存成功，股票代码为SZ002736
第17个文件保存成功，股票代码为SZ300015
第18个文件保存成功，股票代码为SH600999
第19个文件保存成功，股票代码为SH600276
第20个文件保存成功，股票代码为SH601236
第21个文件保存成功，股票代码为SH600109
第22个文件保存成功，股票代码为SZ002624
第23个文件保存成功，股票代码为SH603833
第24个文件保存成功，股票代码为SZ002714
第25个文件保存成功，股票代码为SH601878
第26个文件保存成功，股票代码为SH601162
第27个文件保存成功，股票代码为SH600297
第28个文件保存成功，股票代码为SZ000783
第29个文件保存成功，股票代码为SZ000333
第30个文件保存成功，股票代码为SZ000627
第31个文件保存成功，股票代码为SZ000166
第32个文件保存成功，股票代码为SH601108
第33个文件保存成功，股票代码为SH600030
第34个文件保存成功，股票代码为SZ000661
第35个文件保存成功，股票代码为SH603160
第36个文件保存成功，股票代码为SH601688
第37个文件保存成功，股票代码为SH600837
第38个文件保存成功，股票代码为SH600436
第39个文件保存成功，股票代码为SZ000538
第40个文件保存成功，股票代码为SZ002773
第41个文件保存成

In [ ]:
# 分割股票后获得每个股票下面的unique members